In [2]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('aac_shelter_outcomes.csv')

In [4]:
data.head()

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
0,2 weeks,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male
1,1 year,A666430,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female
2,1 year,A675708,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male
3,9 years,A680386,Dog,Miniature Schnauzer Mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male
4,5 months,A683115,Other,Bat Mix,Brown,2014-01-07T00:00:00,2014-07-07T14:04:00,2014-07-07T14:04:00,NaN,Rabies Risk,Euthanasia,Unknown


Проверю на наличие дубликатов данных

In [5]:
data.duplicated().sum()

10

In [6]:
duplicated = data[data.duplicated(keep=False)]
duplicated.head(10)

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
1240,2 years,A686827,Cat,Domestic Shorthair Mix,Calico,2012-02-27T00:00:00,2014-08-28T09:00:00,2014-08-28T09:00:00,NaN,SCRP,Transfer,Spayed Female
1241,2 years,A686827,Cat,Domestic Shorthair Mix,Calico,2012-02-27T00:00:00,2014-08-28T09:00:00,2014-08-28T09:00:00,NaN,SCRP,Transfer,Spayed Female
5827,4 months,A683782,Cat,Domestic Shorthair Mix,Brown Tabby,2014-02-15T00:00:00,2014-07-16T09:00:00,2014-07-16T09:00:00,NaN,SCRP,Transfer,Neutered Male
5828,4 months,A683782,Cat,Domestic Shorthair Mix,Brown Tabby,2014-02-15T00:00:00,2014-07-16T09:00:00,2014-07-16T09:00:00,NaN,SCRP,Transfer,Neutered Male
12145,1 year,A682781,Cat,Domestic Shorthair Mix,White/Black,2013-01-02T00:00:00,2014-07-03T09:00:00,2014-07-03T09:00:00,NaN,SCRP,Transfer,Neutered Male
12146,1 year,A682781,Cat,Domestic Shorthair Mix,White/Black,2013-01-02T00:00:00,2014-07-03T09:00:00,2014-07-03T09:00:00,NaN,SCRP,Transfer,Neutered Male
17319,3 months,A660948,Cat,Domestic Shorthair Mix,Brown Tabby/White,2013-07-08T00:00:00,2013-11-03T18:16:00,2013-11-03T18:16:00,*Roy,NaN,Adoption,Neutered Male
17320,3 months,A660948,Cat,Domestic Shorthair Mix,Brown Tabby/White,2013-07-08T00:00:00,2013-11-03T18:16:00,2013-11-03T18:16:00,*Roy,NaN,Adoption,Neutered Male
23439,10 months,A695798,Cat,Domestic Shorthair Mix,Orange,2014-02-23T00:00:00,2015-01-23T12:34:00,2015-01-23T12:34:00,Jim,Suffering,Euthanasia,Neutered Male
23440,10 months,A695798,Cat,Domestic Shorthair Mix,Orange,2014-02-23T00:00:00,2015-01-23T12:34:00,2015-01-23T12:34:00,Jim,Suffering,Euthanasia,Neutered Male


In [7]:
data.drop_duplicates(inplace=True)
data.duplicated().sum()

0

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78246 entries, 0 to 78255
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   age_upon_outcome  78238 non-null  object
 1   animal_id         78246 non-null  object
 2   animal_type       78246 non-null  object
 3   breed             78246 non-null  object
 4   color             78246 non-null  object
 5   date_of_birth     78246 non-null  object
 6   datetime          78246 non-null  object
 7   monthyear         78246 non-null  object
 8   name              54365 non-null  object
 9   outcome_subtype   35958 non-null  object
 10  outcome_type      78234 non-null  object
 11  sex_upon_outcome  78244 non-null  object
dtypes: object(12)
memory usage: 7.8+ MB


In [9]:
#удалю animal_id и name так как они не будут нести смысловую нагрузку для модели
data.drop(['animal_id', 'name'], axis=1, inplace=True)
data.head()

,age_upon_outcome,animal_type,breed,color,date_of_birth,datetime,monthyear,outcome_subtype,outcome_type,sex_upon_outcome
0,2 weeks,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,Partner,Transfer,Intact Male
1,1 year,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Partner,Transfer,Spayed Female
2,1 year,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,NaN,Adoption,Neutered Male
3,9 years,Dog,Miniature Schnauzer Mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Partner,Transfer,Neutered Male
4,5 months,Other,Bat Mix,Brown,2014-01-07T00:00:00,2014-07-07T14:04:00,2014-07-07T14:04:00,Rabies Risk,Euthanasia,Unknown


In [10]:
import pandas_profiling

In [11]:
data.profile_report()

#### Перечень проблем, которые требуют решения:
<ol>
<li>удалить age_upon_outcome и monthyear - так как они дублирую информацию</li>
<li>из date_of_birth и datetime сделать новый признак - возраст, после чего удалить date_of_birth</li>
<li>удалить outcome_type и sex_upon_outcome с значениями NaN</li>
<li>сделать новый булиевый признак Mix(помесь породы) из breed, после чего удалить Mix из breed</li>
<li>преобразовать данные в animal_type, outcome_type, sex_upon_outcome с помощью one-hot encoding</li>
 
</ol>

In [12]:
info_reg_exp = r'([0-9]{4}-[0-9]{2}-[0-9]{2})'
data['date_of_birth'] = data['date_of_birth'].str.extract(info_reg_exp)
data['datetime'] = data['datetime'].str.extract(info_reg_exp)

In [13]:
data['age_outcome'] = pd.to_datetime(data['datetime']) - pd.to_datetime(data['date_of_birth'])
data.head()

,age_upon_outcome,animal_type,breed,color,date_of_birth,datetime,monthyear,outcome_subtype,outcome_type,sex_upon_outcome,age_outcome
0,2 weeks,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07,2014-07-22,2014-07-22T16:04:00,Partner,Transfer,Intact Male,15 days
1,1 year,Dog,Beagle Mix,White/Brown,2012-11-06,2013-11-07,2013-11-07T11:47:00,Partner,Transfer,Spayed Female,366 days
2,1 year,Dog,Pit Bull,Blue/White,2013-03-31,2014-06-03,2014-06-03T14:20:00,NaN,Adoption,Neutered Male,429 days
3,9 years,Dog,Miniature Schnauzer Mix,White,2005-06-02,2014-06-15,2014-06-15T15:50:00,Partner,Transfer,Neutered Male,3300 days
4,5 months,Other,Bat Mix,Brown,2014-01-07,2014-07-07,2014-07-07T14:04:00,Rabies Risk,Euthanasia,Unknown,181 days


In [14]:
data.drop(['age_upon_outcome', 'date_of_birth', 'datetime', 'monthyear'], axis=1, inplace=True) #оставляю только возраст 

In [15]:
data.head()

,animal_type,breed,color,outcome_subtype,outcome_type,sex_upon_outcome,age_outcome
0,Cat,Domestic Shorthair Mix,Orange Tabby,Partner,Transfer,Intact Male,15 days
1,Dog,Beagle Mix,White/Brown,Partner,Transfer,Spayed Female,366 days
2,Dog,Pit Bull,Blue/White,NaN,Adoption,Neutered Male,429 days
3,Dog,Miniature Schnauzer Mix,White,Partner,Transfer,Neutered Male,3300 days
4,Other,Bat Mix,Brown,Rabies Risk,Euthanasia,Unknown,181 days


In [27]:
data.loc[:, 'age_outcome'] = data['age_outcome'].astype(str)
data['age_outcome'] = data['age_outcome'].str.extract(r'([0-9]+)')
data.loc[:, 'age_outcome'] = data['age_outcome'].astype(int)
data.head()

,animal_type,breed,color,outcome_subtype,outcome_type,sex_upon_outcome,age_outcome,Mix
0,Cat,Domestic Shorthair,Orange Tabby,Partner,Transfer,Intact Male,15,1
1,Dog,Beagle,White/Brown,Partner,Transfer,Spayed Female,366,1
2,Dog,Pit Bull,Blue/White,NaN,Adoption,Neutered Male,429,0
3,Dog,Miniature Schnauzer,White,Partner,Transfer,Neutered Male,3300,1
4,Other,Bat,Brown,Rabies Risk,Euthanasia,Unknown,181,1


In [22]:
data['Mix'] = [1 if ('Mix' in value) else 0 for value in data['breed']]

In [23]:
data['breed'] = [value.replace(' Mix','') for value in data['breed'] ]
data.head()

,animal_type,breed,color,outcome_subtype,outcome_type,sex_upon_outcome,age_outcome,Mix
0,Cat,Domestic Shorthair,Orange Tabby,Partner,Transfer,Intact Male,15,1
1,Dog,Beagle,White/Brown,Partner,Transfer,Spayed Female,366,1
2,Dog,Pit Bull,Blue/White,NaN,Adoption,Neutered Male,429,0
3,Dog,Miniature Schnauzer,White,Partner,Transfer,Neutered Male,3300,1
4,Other,Bat,Brown,Rabies Risk,Euthanasia,Unknown,181,1


In [28]:
data.describe()

,age_outcome,Mix
count,78246.000000,78246.000000
mean,817.085333,0.841755
std,1077.144733,0.364972
min,0.000000,0.000000
25%,95.000000,1.000000
50%,371.000000,1.000000
75%,1096.000000,1.000000
max,9137.000000,1.000000


In [24]:
oh_enc = OneHotEncoder()
ohe_addr = oh_enc.fit_transform(data[['animal_type']]).toarray()
ohe_addr = pd.DataFrame(ohe_addr, columns=oh_enc.categories_[0])
ohe_addr.head()

,Bird,Cat,Dog,Livestock,Other
0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0


In [59]:
data.drop(['animal_type'], axis=1, inplace=True)
data = pd.concat([data, ohe_addr], axis=1)
data.head()

,breed,color,outcome_subtype,outcome_type,sex_upon_outcome,age_outcome,Mix,Bird,Cat,Dog,Livestock,Other
0,Domestic Shorthair,Orange Tabby,Partner,Transfer,Intact Male,15 days,1.0,0.0,1.0,0.0,0.0,0.0
1,Beagle,White/Brown,Partner,Transfer,Spayed Female,366 days,1.0,0.0,0.0,1.0,0.0,0.0
2,Pit Bull,Blue/White,NaN,Adoption,Neutered Male,429 days,0.0,0.0,0.0,1.0,0.0,0.0
3,Miniature Schnauzer,White,Partner,Transfer,Neutered Male,3300 days,1.0,0.0,0.0,1.0,0.0,0.0
4,Bat,Brown,Rabies Risk,Euthanasia,Unknown,181 days,1.0,0.0,0.0,0.0,0.0,1.0


In [63]:
data.isna().sum()

breed                  10
color                  10
outcome_subtype     42298
outcome_type           22
sex_upon_outcome       12
age_outcome            10
Mix                    10
Bird                   10
Cat                    10
Dog                    10
Livestock              10
Other                  10
dtype: int64

In [65]:
data = data.fillna('Not')  #заменяю nan на слово Not

In [70]:
data = data.query('outcome_type != "Not"') 

In [103]:
cvect = CountVectorizer()
vals = cvect.fit_transform(data['color']).toarray()
cols = sorted(cvect.vocabulary_, key=lambda x: cvect.vocabulary_[x])
data_color = pd.DataFrame(vals, columns=cols)
data_color.head()

,agouti,apricot,black,blue,brindle,brown,buff,calico,chocolate,cream,...,smoke,tabby,tan,tick,tiger,torbie,tortie,tricolor,white,yellow
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [108]:
cvect = CountVectorizer()
vals = cvect.fit_transform(data['sex_upon_outcome']).toarray()
cols = sorted(cvect.vocabulary_, key=lambda x: cvect.vocabulary_[x])
data_sex = pd.DataFrame(vals, columns=cols)
data_sex.head()

,female,intact,male,neutered,not,spayed,unknown
0,0,1,1,0,0,0,0
1,1,0,0,0,0,1,0
2,0,0,1,1,0,0,0
3,0,0,1,1,0,0,0
4,0,0,0,0,0,0,1


In [112]:
data.drop(['sex_upon_outcome'], axis=1, inplace=True)

In [ ]:
data = pd.concat([data, data_sex], axis=1)

In [113]:
data.head()

,breed,color,outcome_subtype,outcome_type,age_outcome,Mix,Bird,Cat,Dog,Livestock,...,not,spayed,unknown,female,intact,male,neutered,not,spayed,unknown
0,Domestic Shorthair,Orange Tabby,Partner,Transfer,15 days 00:00:00,1,0,1,0,0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,Beagle,White/Brown,Partner,Transfer,366 days 00:00:00,1,0,0,1,0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,Pit Bull,Blue/White,Not,Adoption,429 days 00:00:00,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,Miniature Schnauzer,White,Partner,Transfer,3300 days 00:00:00,1,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
4,Bat,Brown,Rabies Risk,Euthanasia,181 days 00:00:00,1,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [115]:
data.isna().sum()

breed              22
color              22
outcome_subtype    22
outcome_type       22
age_outcome        22
Mix                22
Bird               22
Cat                22
Dog                22
Livestock          22
Other              22
female             22
intact             22
male               22
neutered           22
not                22
spayed             22
unknown            22
female             22
intact             22
male               22
neutered           22
not                22
spayed             22
unknown            22
dtype: int64

In [114]:
cvect = CountVectorizer()
vals = cvect.fit_transform(data['outcome_type']).toarray()
cols = sorted(cvect.vocabulary_, key=lambda x: cvect.vocabulary_[x])
outcome_type = pd.DataFrame(vals, columns=cols)
outcome_type.head()

ValueError: np.nan is an invalid document, expected byte or unicode string.